# Day 1

In [38]:
from functools import reduce
from operator import add
a = None

def increasing(b):
    global a
    if a == None:
        result = False
    else:
        result = b > a
    a = b    
    return result


with open('day1_1.txt', 'r') as f:
    g = (int(line.strip()) for line in f)
    g = (increasing(x) for x in g)
    n = reduce(add, g)
print(n)

1645


In [37]:
from itertools import tee

with open('day1_2.txt', 'r') as f, open('day1_1.txt', 'w') as w:
    g = (int(line.strip()) for line in f)
    t = tee(g, 3)
    for i, itr in enumerate(t):
        for _ in range(i):
            next(itr)
    w.writelines(f'{sum(t)}\n' for t in zip(*t))

# Day 2

In [27]:
h_pos = 0
depth = 0

def process_command(line):
    global h_pos, depth
    command, arg = line.split(' ')
    arg = int(arg)
    if command == 'down':
        depth += arg
    elif command == 'up':
        depth -= arg
    else:
        h_pos += arg
        
with open('day2.txt', 'r') as f:
    for line in f:
        process_command(line)
print(h_pos, depth, h_pos * depth)

1845 916 1690020


In [29]:
aim = 0
h_pos = 0
depth = 0

def process_command(line):
    global aim, h_pos, depth
    command, arg = line.split(' ')
    arg = int(arg)
    if command == 'down':
        aim += arg
    elif command == 'up':
        aim -= arg
    else:
        h_pos += arg
        depth += arg * aim
        
with open('day2.txt', 'r') as f:
    for line in f:
        process_command(line)
print(h_pos, depth, h_pos * depth)

1845 763408 1408487760


# Day 3

In [1]:
def process_line(line, r_ones, r_zeroes, lines):
    lines.append(line)
    for i in range(len(line)):
        n = int(line[i])
        r_ones[i] += n
        r_zeroes[i] += int(not n)
        
lines = []

with open('day3.txt', 'r') as f:
    line = next(f).strip()
    n = len(line)
    r_ones = [0] * n
    r_zeroes = [0] * n
    process_line(line, r_ones, r_zeroes, lines)
    for line in f:
        process_line(line.strip(), r_ones, r_zeroes, lines)

gamma = [int(a >= b) for a, b in zip(r_ones, r_zeroes)]
epsilon = [int(not a) for a in gamma]
gamma = f'0b{"".join(str(n) for n in gamma)}'
epsilon = f'0b{"".join(str(n) for n in epsilon)}'
gamma = eval(gamma)
epsilon = eval(epsilon)
print(gamma, epsilon, gamma * epsilon)

844 3251 2743844


In [2]:
def check(digit, n_ones, n_zeroes, isOx):
    if isOx:
        return ((digit == '1') and n_ones >= n_zeroes) or ((digit == '0') and n_ones < n_zeroes)
    else:
        return ((digit == '0') and n_ones >= n_zeroes) or ((digit == '1') and n_ones < n_zeroes)

def find_gas(lines, n_ones, n_zeroes, isOx):
    index = 0
    while True:        
        lines = [line for line in lines if check(line[index], n_ones, n_zeroes, isOx)]
        if len(lines) == 1:            
            return eval(f'0b{lines[0]}')
        index += 1
        n_ones = sum(int(line[index]) for line in lines)
        n_zeroes = sum(not int(line[index]) for line in lines)

o, c =[find_gas(lines, r_ones[0], r_zeroes[0], isOx) for isOx in (True, False)] 
print(o, c, o*c)

1981 3371 6677951


# Day 4

In [163]:
from itertools import islice, chain
import re

class Board:
    
    regexp = re.compile(r'\s+')
    
    @classmethod    
    def _format_el(cls, el, marked):
        if marked:
            return f'({el})'
        else:
            return el
    
    def __init__(self, lines):
        self.rows = [
            {int(el): False for el in Board.regexp.split(line)}
            for line in lines
        ]
        self.sidelen = len(self.rows)        
        self.win = False
        self.score = 0
        
    def play(self, n):
        had_n = False
        for row in self.rows:
            if n in row:
                row[n] = True
                had_n = True
        if had_n:
            self._check_win()
            if self.win:
                self._calc_score(n)
        return self.win
        
    def __bool__(self):
        return bool(self.sidelen)
    
    def __str__(self):
        arr = []
        arr.append(f'win: {self.win}, score: {self.score}')
        arr.extend(
            '\t'.join(f'{self._format_el(el, marked):^4}' for el, marked in row.items())
            for row in self.rows
        )
        return '\n'.join(arr)
            
    def _calc_score(self, n):
        self.score = sum(
            sum(k for k, v in row.items() if not v)
            for row in self.rows
        ) * n
    
    def _check_win(self):
        if not self.win:
            self.win = any(
                all(row.values())
                for row in self.rows
            )
        if not self.win:
            self.win = any(
                all(col)
                for col in zip(*[row.values() for row in self.rows])
            )

def board_generator(g):
    while True:
        result = Board(islice(g, 0, 5))
        if result:
            yield result
        else:
            break
    
def play_number(n, boards):
    boards = (board for board in boards if not board.win)
    return [
        board.score
        for board in boards
        if board.play(n)
    ]    

with open('day4.txt', 'r') as f:
    g = (line.strip() for line in f)
    line = next(g).split(',')
    inp_arr = [int(el) for el in line]
    g = (line for line in g if line)
    boards = [*board_generator(g)]
scores = chain.from_iterable(
    play_number(n, boards)
    for n in inp_arr
)
scores = [score for score in scores if score]
print(scores[0], scores[-1])

55770 2980


In [ ]:
# For checking that each number is unique in a row

regexp = re.compile(r'\s+')
with open('day4.txt', 'r') as f:
    g = (line.strip() for line in f)
    next(g)    
    g = (set(regexp.split(line)) for line in g if line)
    assert all(len(line) == 5 for line in g)

# Day 5

In [35]:
from collections import namedtuple
from itertools import chain

LineOfVents = namedtuple('LineOfVents', 'x0 y0 x1 y1')
def parse_line(line):
    g = (el.strip().split(',') for el in line.split('->'))
    g = (
        (int(el) for el in t)
        for t in g
    )
    (x0, y0), (x1, y1) = g
    return LineOfVents(x0, y0, x1, y1)

def minmax(a, b):
    return min(a, b), max(a, b)

def draw_point(x, y, canvas):    
    if canvas[y][x] == '.':
        canvas[y][x] = 1
    else:
        canvas[y][x] += 1
        
def draw_line(line, canvas):
    if line.y0 == line.y1:
        x0, x1 = minmax(line.x0, line.x1)
        y = line.y0
        for x in range(x0, x1+1):
            draw_point(x, y, canvas)
    elif line.x0 == line.x1:
        x = line.x0
        y0, y1 = minmax(line.y0, line.y1)
        for y in range(y0, y1+1):
            draw_point(x, y, canvas)
    else:
        x0, x1 = minmax(line.x0, line.x1)
        length = x1 - x0
        x0, y0, x1, y1 = line
        xc = 1 if x1 > x0 else -1
        yc = 1 if y1 > y0 else -1
        x, y = x0, y0
        for _ in range(length+1):
            draw_point(x, y, canvas)
            x += xc
            y += yc

with open('day5.txt', 'r') as f:
    n = max(max(parse_line(line)) for line in f)

print('max coord:', n)

canvas = [
    # ['.'] * (n+1)
    [0] * (n+1)
    for _ in range(n+1)
]
    
with open('day5.txt', 'r') as f:
    g = (parse_line(line) for line in f)
    # g = (v for v in g if (v.x0 == v.x1) or (v.y0 == v.y1))
    for line in g:
        draw_line(line, canvas)
c = chain.from_iterable(canvas)
print('number of intersections:', len([p for p in c if p > 1]))
# for row in canvas:
#     print(' '.join(str(el) for el in row))

max coord: 990
number of intersections: 19349


# Day 6

In [333]:
# Modeling approach -- works for part 1, but too time-consuming for part 2

days_passed = 0

def pass_day(fishes):
    global days_passed
    requires_printing = False
    days_passed += 1
    new_fishes = []
    for i, fish in enumerate(fishes):
        fish -= 1
        if fish < 0:
            fish = 6
            new_fishes.append(8)
            requires_printing = True
        fishes[i] = fish
    fishes.extend(new_fishes)
    # if requires_printing:
    #     print(days_passed, fishes)
    

def pass_days(days, fishes):
    for _ in range(days):
        pass_day(fishes)

with open('day6.txt', 'r') as f:
    fishes = [int(x) for x in f.read().strip().split(',')]

# fishes = [3]
        
pass_days(18, fishes)
# pass_days(80, fishes)
# pass_days(256, fishes)
# print(fishes)
print(len(fishes))
# print(fishes)

26


In [ ]:
# Calculating approach -- same problem as the modeling approach

def direct_children(days_till_new_child, days_total):
    # because the child would be created after 0, when the timer gets reset to 6
    days_till_new_child += 1    
    if days_total < days_till_new_child:
        # The fish won't be able to make any children in such a short amount of days
        return 0
    else:
        # The fish would make 1 child + a child per each 7 days until the end of the period
        return 1 + (days_total - days_till_new_child) // 7
    
def total_children(days_till_new_child, days_total):
    dc = direct_children(days_till_new_child, days_total)
    if dc == 0:
        return 0
    return dc + sum(
        total_children(
            days_till_new_child = 8,
            days_total = days_total - ( (days_till_new_child + 1) + (7 * i) )            
        )
        for i in range(dc)
    )
    
with open('day6.txt', 'r') as f:
    fishes = [int(x) for x in f.read().strip().split(',')]
    
s = set(fishes)
for fish in s:
    print(total_children(fish, 256))
# len(fishes) + sum(total_children(fish, 256) for fish in fishes)

In [41]:
# Works for both parts

from collections import Counter

def pass_day(fishes):
    n = fishes[0]
    fishes[0] = 0
    for i in range(1, 9):        
        fishes[i-1] += fishes[i]
        fishes[i] = 0
    fishes[8] += n
    fishes[6] += n
    
def pass_days(days, fishes):
    for _ in range(days):
        pass_day(fishes)

with open('day6.txt', 'r') as f:
    fishes = [int(x) for x in f.read().strip().split(',')]

fishes = Counter(fishes)
pass_days(256, fishes)
sum(fishes.values())

1732821262171

# Day 7

In [16]:
def align(positions, target):
    return sum(abs(target - position) for position in positions)

def align_optimal(positions):
    results = {
        align(positions, i): i
        for i in range(min(positions), max(positions) + 1)
    }
    optimal_fuel = min(results)
    return results[optimal_fuel], optimal_fuel

with open('day7.txt', 'r') as f:
    positions = [int(el) for el in f.read().strip().split(',')]

print(align_optimal(positions))

(2, 37)


In [25]:
def align(positions, target):
    return sum(abs(target - position) * (1 + abs(target - position)) // 2 for position in positions)

def align_optimal(positions):
    results = {
        align(positions, i): i
        for i in range(min(positions), max(positions) + 1)
    }
    optimal_fuel = min(results)
    return results[optimal_fuel], optimal_fuel

with open('day7.txt', 'r') as f:
    positions = [int(el) for el in f.read().strip().split(',')]

print(align_optimal(positions))

(484, 93397632)
